In [1]:
# Import packages
import pandas as pd
import ast 
import numpy as np
import json
import re
import os

data_dir = os.path.join("..", "data")
data_path = os.path.join(data_dir, "tpb_df.csv")

In [3]:
# regex patterns

geolist = ["lebanon", "syria", "jordan", "iraq", "greece", "turkey", "cyprus", "mediterranean", "EU"]
wordlist = ["migrant", "refugee", "transit", "displacement", "border", "return", "pushback", "boat", "drowning", "hunger"]

regex_geo = re.compile("|".join([f"\\b{geo}\\b" for geo in geolist]), re.IGNORECASE)
regex_words = re.compile("|".join([f"\\b{word}" for word in wordlist]), re.IGNORECASE)
regex_hashtags = re.compile("|".join([f"\\b#{word}" for word in wordlist]), re.IGNORECASE)

In [5]:
regex_words

re.compile(r'\bmigrant|\brefugee|\btransit|\bdisplacement|\bborder|\breturn|\bpushback|\bboat|\bdrowning|\bhunger',
re.IGNORECASE|re.UNICODE)

In [2]:
df = pd.read_csv(data_path)
df.head()

,created_at,text,id,author_id,referenced_tweets,retweet_count,reply_count,like_count,quote_count,hashtags,...,description,verified,name,url,username,author_created_at,followers_count,following_count,tweet_count,listed_count
0,2021-12-07T23:50:06.000Z,A U.S. judge on Tuesday blocked the Biden admi...,1468367242064760841,3108351,NaN,41,12,111,3,[],...,Sign up for our newsletters and email alerts: ...,True,The Wall Street Journal,https://t.co/GhhR6PLfem,WSJ,2007-04-01T06:22:13.000Z,19263346,1046,357831,122904
1,2021-12-07T23:18:46.000Z,Analysis: The wide and dangerous gap between T...,1468359359080054784,2467791,NaN,44,33,114,1,[],...,Democracy Dies in Darkness,True,The Washington Post,https://t.co/1KN78z0bbe,washingtonpost,2007-03-27T11:19:39.000Z,18516470,1715,412041,2
2,2021-12-07T23:00:47.000Z,Why are many pregnant women hesitant to take t...,1468354833719697427,4970411,NaN,16,5,34,1,[],...,Hear the human story and join the discussion. ...,True,Al Jazeera English,https://t.co/fDSF1GNVXl,AJEnglish,2007-04-17T08:23:08.000Z,7292176,238,293680,54442
3,2021-12-07T22:15:08.000Z,While much is still unknown about the Omicron ...,1468343345642618884,807095,NaN,59,94,192,7,[],...,News tips? Share them here: https://t.co/ghL9O...,True,The New York Times,http://t.co/ahvuWqicF9,nytimes,2007-03-02T20:41:42.000Z,51136117,860,455829,211884
4,2021-12-07T21:56:04.000Z,6/ What has played out in Lower Manhattan amon...,1468338544582316037,807095,"[{'type': 'replied_to', 'id': '146833854276188...",3,1,10,0,[],...,News tips? Share them here: https://t.co/ghL9O...,True,The New York Times,http://t.co/ahvuWqicF9,nytimes,2007-03-02T20:41:42.000Z,51136117,860,455829,211884


In [8]:
# create subset

#df = pd.read_csv(data_path)

df_sub = df.loc[df['text'].apply(lambda t: (bool(regex_words.search(t)) or bool(regex_hashtags.search(t))) and bool(regex_geo.search(t))), :]
df_sub['id'] = df_sub['id'].astype('str')
df_sub['tweet link'] = "https://twitter.com/" + df_sub['username'] + "/status/" + df_sub['id']

keep_columns = ['created_at', 'name', 'username', 'text', 'tweet link', 'id', 'hashtags', 'urls', 'retweet_count', 
                'reply_count', 'like_count', 'quote_count']

df_sub = df_sub.loc[:, keep_columns]

<ipython-input-8-ee00ea87a3ad>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['id'] = df_sub['id'].astype('str')
<ipython-input-8-ee00ea87a3ad>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['tweet link'] = "https://twitter.com/" + df_sub['username'] + "/status/" + df_sub['id']


In [9]:
# comparison 

print(df.shape,
      df_sub.shape)

(39049, 23) (1414, 12)


In [10]:
df_sub.head()

,created_at,name,username,text,tweet link,id,hashtags,urls,retweet_count,reply_count,like_count,quote_count
22,2021-12-07T15:19:58.000Z,Al Jazeera English,AJEnglish,RT @AJEnglish: Libyan authorities get support ...,https://twitter.com/AJEnglish/status/146823886...,1468238862770216960,[],[],60,0,0,0
23,2021-12-07T14:25:45.000Z,Al Jazeera English,AJEnglish,"""Funds from the EU and member states, sometime...",https://twitter.com/AJEnglish/status/146822522...,1468225220125331467,['AJOpinion'],['https://t.co/1lv9uCE73H'],18,4,40,1
44,2021-12-07T08:45:00.000Z,Al Jazeera English,AJEnglish,Libyan authorities get support from the EU to ...,https://twitter.com/AJEnglish/status/146813946...,1468139466946535425,[],['https://t.co/KfrjPxz5du'],60,11,114,4
45,2021-12-07T08:38:00.000Z,InfoMigrants,InfoMigrants,Some 11 members of the Afghan boxing team and ...,https://twitter.com/InfoMigrants/status/146813...,1468137705049571328,[],['https://t.co/qxamPZ4U2J'],7,0,13,1
54,2021-12-07T03:15:00.000Z,Al Jazeera English,AJEnglish,"“Now, supermarkets have become boutiques.”\n\n...",https://twitter.com/AJEnglish/status/146805641...,1468056419417088003,[],['https://t.co/fzjWcNV8Az'],19,5,35,2


In [11]:
# export

df_sub.to_excel(os.path.join(data_dir, "tpb_tweets_simple-filter.xlsx"), index = False)